# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [2]:
city_data= pd.read_csv('../output_data/cities.csv')
city_data.head()

,City Name,Latitude,Longtitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Orlik,52.5178,99.8250,19.41,46,100,0.94,RU,1659957415
1,Sunbury,-37.5833,144.7333,7.90,78,20,0.45,AU,1659957416
2,Nelson Bay,-32.7167,152.1500,14.49,77,11,4.92,AU,1659957417
3,Merauke,-8.4667,140.3333,25.59,77,40,7.78,ID,1659957417
4,Port Alfred,-33.5906,26.8910,17.88,90,91,3.13,ZA,1659957418


### Humidity Heatmap

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Set the locations and weight
locations = city_data[["Latitude","Longtitude"]].astype(float)
humidity = city_data["Humidity"].astype(float)

# Create a Humidity Heatmap
fig = gmaps.figure(center=(10,0.00),zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [5]:
hotel_df = city_data.loc[(city_data["Humidity"]<80) &
                         (city_data["Humidity"]>20) &
                             (city_data["Max Temperature"]<35) &
                            (city_data["Max Temperature"]>20) &
                            (city_data["Cloudiness"]<70) &
                            (city_data["Wind Speed"]<5)&
                        (city_data["Wind Speed"]>1)&
                        (city_data["Country"] != "AU")]
len(hotel_df)

64

In [6]:
hotel_df.head()

,City Name,Latitude,Longtitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
8,Veroli,41.6911,13.4177,34.06,34,40,2.57,IT,1659957420
15,Bambous Virieux,-20.3428,57.7575,24.56,64,20,4.12,MU,1659957423
16,Ponta do Sol,32.6667,-17.1000,26.67,72,4,1.53,PT,1659957424
35,Kelo,9.3086,15.8066,29.48,63,61,3.32,TD,1659957432
49,Hilo,19.7297,-155.0900,25.71,74,40,3.60,US,1659957440


### Hotel Map

In [7]:
# Add column for hotel to the DataFrame
hotel_df["Hotel Name"] = ""

# Params dictionary to search for hotel
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

# Using the the lat/long to identify in which city to look for
for index, row in hotel_df.iterrows():
    #Get each cities lat & long and add to api search
    lat = row["Latitude"]
    lng = row["Longtitude"]
    params["location"] = f"{lat},{lng}"

    # Request url
    hotel_name = requests.get(base_url, params=params)
    hotel_name =  hotel_name.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][1]["name"]
        print("Found hotel for " + hotel_name["results"][0]["name"])
    except (KeyError, IndexError):
        print("Missing result... skipping")

C:\Users\Carol's PC\AppData\Local\Temp\ipykernel_9996\1250253230.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


Found hotel for Veroli
Found hotel for Vieux Grand Port
Found hotel for Ribeira Brava
Found hotel for Kelo
Found hotel for Hilo
Found hotel for Mahón
Found hotel for Portland
Found hotel for Iki-Burul
Found hotel for Coimbra
Found hotel for San Román de Cameros
Found hotel for Newport
Found hotel for St. George
Found hotel for Musholla Babut Taubat
Found hotel for Avarua District
Found hotel for Jammu
Found hotel for Stade
Found hotel for Shubarkuduk
Found hotel for Praia da Vitória
Found hotel for Prado
Found hotel for Krasnovishersk
Found hotel for Ponta Delgada
Found hotel for Gerede
Found hotel for Mahebourg
Found hotel for Quzhou
Found hotel for Luderitz
Found hotel for Kingman
Found hotel for Goodlands
Found hotel for Padang
Found hotel for Saint-Paul
Found hotel for Sinazongwe
Found hotel for Bideford
Found hotel for Varzea da Palma
Found hotel for Les Cayes
Found hotel for Namibe
Found hotel for Muros
Found hotel for Margate
Found hotel for La Asuncion
Found hotel for Wincheste

In [8]:
hotel_df.head()

,City Name,Latitude,Longtitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,Veroli,41.6911,13.4177,34.06,34,40,2.57,IT,1659957420,Hotel Relais Filonardi
15,Bambous Virieux,-20.3428,57.7575,24.56,64,20,4.12,MU,1659957423,La Case du Pêcheur
16,Ponta do Sol,32.6667,-17.1000,26.67,72,4,1.53,PT,1659957424,Hotel do Campo
35,Kelo,9.3086,15.8066,29.48,63,61,3.32,TD,1659957432,Centro Educativo Charles Lwanga (CECL) - Misio...
49,Hilo,19.7297,-155.0900,25.71,74,40,3.60,US,1659957440,Hilo Hawaiian Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longtitude"]]

In [10]:
# Set the locations and weight
locations = hotel_df[["Latitude","Longtitude"]].astype(float)
humidity = hotel_df["Humidity"].astype(float)

# Create a Humidity Heatmap
fig = gmaps.figure(center=(10,0.00),zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

# Add marker layer ontop of heat map
hotel_marker = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(hotel_marker)

fig

Figure(layout=FigureLayout(height='420px'))